[https://huggingface.co/BAAI/bge-reranker-large](https://huggingface.co/BAAI/bge-reranker-large)

# 下载模型

获取 hugging face access token：[https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

# 加载环境变量
load_dotenv(find_dotenv())
HUGGING_FACE_ACCESS_TOKEN = os.getenv('HUGGING_FACE_ACCESS_TOKEN')

In [3]:
from huggingface_hub import snapshot_download

repo_id = "BAAI/bge-reranker-large"  # 模型在huggingface上的名称
local_dir = f"{repo_id.split('/')[0]}/{repo_id.split('/')[1]}"  # 本地模型存储的地址
local_dir_use_symlinks = False  # 本地模型使用文件保存，而非blob形式保存

# 注意：在hugging face上生成的自己的access token，否则模型下载会中断
token = HUGGING_FACE_ACCESS_TOKEN

# 开始下载
snapshot_download(
    repo_id=repo_id,
    local_dir=local_dir,
    local_dir_use_symlinks=local_dir_use_symlinks,
    token=token,
)

D:\Software\anaconda3\envs\py38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 11 files:   0%|                                                                                                                 | 0/11 [00:00<?, ?it/s]
model.safetensors:   0%|                                                                                                            | 0.00/2.24G [00:00<?, ?B/s]


config.json:   0%|                                                                                                                    | 0.00/801 [00:00<?, ?B/s]

.gitattributes: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 1.63k/1.63k [00:00<00:00, 467kB/s]
Fetching 11 files:   9%|█████████▌                                                                

'D:\\Work\\GreedyAI\\Workspace\\AdvancedRAG\\BAAI\\bge-reranker-large'

# 测试模型

## Usage for Reranker

### Using Huggingface transformers

In [1]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-large')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-large')
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)


D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([-5.6085,  5.7623])


In [2]:
scores

tensor([-5.6085,  5.7623])

In [3]:
results = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
results

[(1, tensor(5.7623)), (0, tensor(-5.6085))]

### Using Sentence-Transformers

In [4]:
from sentence_transformers import CrossEncoder

model_name = 'BAAI/bge-reranker-large'
model = CrossEncoder(model_name)
pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
model.predict(pairs)

array([0.00365301, 0.9968658 ], dtype=float32)